In [58]:
from google.colab import drive
drive.flush_and_unmount()


In [59]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [61]:
from langchain.vectorstores import FAISS

In [62]:
# Import the necessary classes from your uploaded .py files
from embeddings import CodeBERTEmbedder
from vectorstore import VectorStore
from retriever import RAGRetriever
from chatbot import CPChatbot

In [63]:
!pip install -q faiss-cpu
!pip install -q langchain_community
!pip install -q langchain-huggingface

In [64]:
emb=CodeBERTEmbedder()

If you want to use `RobertaLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of RobertaForCausalLM were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['lm_head.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [65]:
from langchain.embeddings import HuggingFaceEmbeddings

# Instead of assigning to `model`, use `model_name`
embedder = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2",
)

In [66]:
vector_store = VectorStore()
# vector_store.create_store(embedder)

In [106]:
retriever = RAGRetriever(embedder, vector_store)

In [69]:
system_message = "You are helping with competitive programming problem solutions."
chatbot = CPChatbot(retriever, system_message)

Device set to use cpu


In [103]:
chtbot=CPChatbot(retriever,system_message)

Device set to use cpu


In [104]:
query = "How do I solve problem C from Contest#792?"
response = chtbot.chat(query)
print(f"Chatbot Response: {response}")

System Message: You are helping with competitive programming problem solutions.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

There are n participants in the contests, with Kevin being the 1-st participant. The i-th participant has a rating of ai. During the contests, each participant solves all problems with a difficulty not exceeding their rating, meaning the i-th participant solves the j-th problem if and only if ai≥bj. In each contest, Kevin's rank is one plus the number of participants who solve more problems than

Hint 2 
 The problem reduces to the following: Given a 1s, b 2s, and c 0s, how many ways are there to build a permutation on these objects so that each cycle sums to a multiple of 3? Let f(a,b,c) be the answer. Write some dynamic programming recurrences for f(a,b,c). 
Hint 3

2048D - Kevin and Competition MemoriesRead all the hints.First, remove all contestants with a rating lower than yours, making you the contestant with the lowes